# SOLVE-GP: A Toy Example

The aim of this example is to illustrate the training process of SOLVE-GP.
The ideal use case of the algorithm should be large-scale applications where the number of inducing points is limited by the computational budget.

In [1]:
import os
# os.environ['PATH'] = os.environ['PATH'] + ':/usr/local/texlive/2019/bin/x86_64-darwin'
import tensorflow as tf
import numpy as np
import gpflow
from gpflow import settings, features, transforms, kullback_leiblers
from gpflow.params import Parameter, Minibatch, DataHolder
from gpflow.conditionals import Kuu, Kuf
from gpflow.decors import autoflow
from scipy.cluster.vq import kmeans2
import matplotlib as mpl
mpl.use("pgf")
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import rc
from matplotlib.backends.backend_pgf import FigureCanvasPgf
mpl.backend_bases.register_backend('pdf', FigureCanvasPgf)
rc('text', usetex=True)
rc("pgf", rcfonts=False, preamble=r'\usepackage{color}')
tf.set_random_seed(1234)
np.random.seed(1234)
import imageio
import moviepy.editor as mp

/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

We implement the SOLVE-GP algorithm as a GPflow class. Please use GPflow version 1.5.1 and Tensorflow 1.x.

In [2]:
class SOLVEGP(gpflow.models.GPModel):
    def __init__(self, X, Y, kern, likelihood, z, a,
                 mean_function=None,
                 num_outputs=None,
                 whiten=True,
                 minibatch_size=None,
                 num_data=None,
                 **kwargs):
        # sort out the X, Y into MiniBatch objects if required.
        if minibatch_size is None:
            X = DataHolder(X)
            Y = DataHolder(Y)
        else:
            X = Minibatch(X, batch_size=minibatch_size, seed=0)
            Y = Minibatch(Y, batch_size=minibatch_size, seed=0)

        # init the super class, accept args
        gpflow.models.GPModel.__init__(
            self, X, Y, kern, likelihood, mean_function, num_outputs, **kwargs)
        self.num_outputs = num_outputs
        self.num_data = num_data or X.shape[0]
        self.whiten = whiten

        # init variational parameters
        self.z = z
        self.a = a

        with gpflow.params_as_tensors_for(self.z), gpflow.params_as_tensors_for(self.a):
            self.Kuu = Kuu(self.z, self.kern, jitter=settings.jitter)
            self.Lu = tf.cholesky(self.Kuu)
            self.Kvv = Kuu(self.a, self.kern, jitter=settings.jitter)
            self.Kuv = self.kern.K(self.z.Z, self.a.Z)
        self.Lu_inv_Kuv = tf.matrix_triangular_solve(self.Lu, self.Kuv)
        self.Cvv = self.Kvv - tf.matmul(
            self.Lu_inv_Kuv, self.Lu_inv_Kuv, transpose_a=True)
        self.Lv = tf.cholesky(self.Cvv)

        # init variational parameters
        M = len(self.z)
        with tf.variable_scope(self.name + "/z"):
            self.qu_mu = Parameter(
                np.zeros((M, num_outputs), dtype=settings.float_type), name='qu_mu')
            if not self.whiten:
                Lu_ = self.enquire_session().run(self.Lu)
                qu_sqrt = np.tile(Lu_[None, :, :], [num_outputs, 1, 1])
            else:
                qu_sqrt = np.tile(np.eye(M, dtype=settings.float_type)[None, :, :], 
                                    [num_outputs, 1, 1])
            self.qu_sqrt = Parameter(
                qu_sqrt, transform=transforms.LowerTriangular(M, num_outputs), name='qu_sqrt')

        M2 = len(self.a)
        with tf.variable_scope(self.name + "/a"):
            self.qv_mu = Parameter(
                np.zeros((M2, num_outputs), dtype=settings.float_type), name='qv_mu')
            if not self.whiten:
                Lv_ = self.enquire_session().run(self.Lv)
                qv_sqrt = np.tile(Lv_[None, :, :], [num_outputs, 1, 1])
            else:
                qv_sqrt = np.tile(np.eye(M2, dtype=settings.float_type)[None, :, :], 
                                    [num_outputs, 1, 1])
            self.qv_sqrt = Parameter(
                qv_sqrt, transform=transforms.LowerTriangular(M2, num_outputs), name='qv_sqrt')
    
    def kl_u(self):
        if self.whiten:
            K = None
        else:
            K = self.Kuu
        return kullback_leiblers.gauss_kl(self.qu_mu, self.qu_sqrt, K)

    def kl_v(self):
        if self.whiten:
            K = None
        else:
            K = self.Cvv
        return kullback_leiblers.gauss_kl(self.qv_mu, self.qv_sqrt, K)

    @autoflow((settings.float_type, [None, None]))
    def predict_f(self, Xnew):
        mean_func, fin, fperp, var_in, var_perp = self._build_predict(Xnew)
        fmean = fin + fperp + mean_func
        fvar = var_in + var_perp
        return fmean, fvar
    
    @autoflow((settings.float_type, [None, None]))
    def predict_fin(self, Xnew):
        mean_func, fin, fperp, var_in, var_perp = self._build_predict(Xnew)
        return fin, var_in

    @autoflow((settings.float_type, [None, None]))
    def predict_fperp(self, Xnew):
        mean_func, fin, fperp, var_in, var_perp = self._build_predict(Xnew)
        return fperp, var_perp

    @gpflow.params_as_tensors
    def build_prior_KL(self):
        return self.kl_u() + self.kl_v()

    @gpflow.params_as_tensors
    def _build_likelihood(self):
        KL = self.build_prior_KL()
        mean_func, fin, fperp, var_in, var_perp = self._build_predict(
            self.X, full_cov=False, full_output_cov=False)
        fmean = fin + fperp + mean_func
        fvar = var_in + var_perp
        var_exp = self.likelihood.variational_expectations(fmean, fvar, self.Y)
        scale = tf.cast(self.num_data, settings.float_type) / tf.cast(
            tf.shape(self.X)[0], settings.float_type)
        likelihood = tf.reduce_sum(var_exp) * scale - KL
        return likelihood

    @gpflow.params_as_tensors
    def _build_predict(self, X, full_cov=False, full_output_cov=False):
        Kuf_ = Kuf(self.z, self.kern, X)
        Kvf = Kuf(self.a, self.kern, X)
        Lu_inv_Kuf = tf.matrix_triangular_solve(self.Lu, Kuf_)
        if not self.whiten:
            Au = tf.matrix_triangular_solve(
                tf.transpose(self.Lu), Lu_inv_Kuf, lower=False)
        else:
            Au = Lu_inv_Kuf
        Cvf = Kvf - tf.matmul(self.Lu_inv_Kuv, Lu_inv_Kuf, transpose_a=True)

        if full_cov:
            Kff = self.kern.K(X)
            Cff = Kff - tf.matmul(Lu_inv_Kuf, Lu_inv_Kuf, transpose_a=True)
        else:
            Kff = self.kern.Kdiag(X)
            Cff = Kff - tf.reduce_sum(tf.square(Lu_inv_Kuf), axis=0)

        Lv_inv_Cvf = tf.matrix_triangular_solve(self.Lv, Cvf)
        if not self.whiten:
            Av = tf.matrix_triangular_solve(
                tf.transpose(self.Lv), Lv_inv_Cvf, lower=False)
        else:
            Av = Lv_inv_Cvf

        fin = tf.matmul(Au, self.qu_mu, transpose_a=True)
        fperp = tf.matmul(Av, self.qv_mu, transpose_a=True)

        # [K, M, M]
        Lqu = tf.matrix_band_part(self.qu_sqrt, -1, 0)
        Au_tiled = tf.tile(Au[None, ...], [self.num_outputs, 1, 1])
        # [K, M, N]
        LquTA = tf.matmul(Lqu, Au_tiled, transpose_a=True)
        if full_cov:
            var_in = tf.matmul(LquTA, LquTA, transpose_a=True)
        else:
            var_in = tf.reduce_sum(tf.square(LquTA), axis=1)
        var_in = tf.transpose(var_in)

        Lqv = tf.matrix_band_part(self.qv_sqrt, -1, 0)
        Av_tiled = tf.tile(Av[None, ...], [self.num_outputs, 1, 1])
        LqvTA = tf.matmul(Lqv, Av_tiled, transpose_a=True)
        if full_cov:
            var_perp = (tf.matmul(LqvTA, LqvTA, transpose_a=True) + Cff -
                        tf.matmul(Lv_inv_Cvf, Lv_inv_Cvf, transpose_a=True))
        else:
            var_perp = (tf.reduce_sum(tf.square(LqvTA), 1) + Cff -
                        tf.reduce_sum(tf.square(Lv_inv_Cvf), 0))
        var_perp = tf.transpose(var_perp)
        return self.mean_function(X), fin, fperp, var_in, var_perp

## Load data

In [3]:
class ToyData1D(object):
    def __init__(self, train_x, train_y, test_x, normalize=False, 
                 dtype=np.float64):
        self.train_x = np.array(train_x, dtype=dtype)[:, None]
        self.train_y = np.array(train_y, dtype=dtype)[:, None]
        self.n_train = self.train_x.shape[0]
        self.test_x = np.array(test_x, dtype=dtype)[:, None]
        self.x_min = np.min(test_x)
        self.x_max = np.max(test_x)
        self.n_test = self.test_x.shape[0]
        if normalize:
            self.normalize()

    def normalize(self):
        self.mean_x = np.mean(self.train_x, axis=0, keepdims=True)
        self.std_x = np.std(self.train_x, axis=0, keepdims=True) + 1e-6
        self.mean_y = np.mean(self.train_y, axis=0, keepdims=True)
        self.std_y = np.std(self.train_y, axis=0, keepdims=True) + 1e-6

        for x in [self.train_x, self.test_x]:
            x -= self.mean_x
            x /= self.std_x

        for x in [self.x_min, self.x_max]:
            x -= self.mean_x.squeeze()
            x /= self.std_x.squeeze()

        self.train_y -= self.mean_y
        self.train_y /= self.std_y

    
def load_snelson_data(n=100, dtype=np.float64):
    def _load_snelson(filename):
        with open(os.path.join("data", "snelson", filename), "r") as f:
            return np.array([float(i) for i in f.read().strip().split("\n")],
                            dtype=dtype)

    train_x = _load_snelson("train_inputs")
    train_y = _load_snelson("train_outputs")
    test_x = _load_snelson("test_inputs")
    perm = np.random.permutation(train_x.shape[0])
    train_x = train_x[perm][:n]
    train_y = train_y[perm][:n]
    return ToyData1D(train_x, train_y, test_x=test_x)

In [4]:
toy = load_snelson_data(n=100)
train_x, train_y = toy.train_x, toy.train_y
test_x = toy.test_x

The dataset has 100 training points, 301 test points.

In [5]:
train_x.shape, train_y.shape, test_x.shape

((100, 1), (100, 1), (301, 1))

## Initialize inducing points and optimize the lower bound



In [6]:
# the number of inducing points
M = 5
# the number of orthogonal inducing points
M2 = 5

# training parameters
batch_size = 20
learning_rate = 0.003
n_iters = 10000

We run K-means on the training points and use the cluster centers to initialize inducing points. Then the SOLVE-GP lower bound is maximized by the ADAM optimizer. The variational parameters and inducing point locations will be updated by gradients.

In [7]:
N, x_dim = train_x.shape
kernel = gpflow.kernels.RBF(x_dim, ARD=False, name="rbf")
likelihood = gpflow.likelihoods.Gaussian(variance=0.1)
inducing_points, _ = kmeans2(train_x, M + M2, minit="points")
with tf.variable_scope("z"):
    z = features.inducingpoint_wrapper(None, inducing_points[:M])
with tf.variable_scope("a"):
    a = features.inducingpoint_wrapper(None, inducing_points[M:])
solvegp = SOLVEGP(train_x, train_y, kernel, likelihood, z, a,
                  num_outputs=1, whiten=False, num_data=N, minibatch_size=batch_size)
obj = solvegp.objective
with gpflow.params_as_tensors_for(likelihood):
    likelihood_var = likelihood.variance

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().


Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().


In [8]:
sess = solvegp.enquire_session()
feed_dict = {} if solvegp.feeds is None else solvegp.feeds
opt = tf.train.AdamOptimizer(learning_rate=learning_rate)
var_list = list(set(solvegp.trainable_tensors))
infer_op = opt.minimize(obj, var_list=var_list)
solvegp.initialize(session=sess)
sess.run(tf.variables_initializer(var_list=opt.variables()))

Some plotting code:

In [9]:
def plot_posterior(ax, y_mean, y_var):
    y_std = np.sqrt(y_var)
    y_upper, y_lower = y_mean + 3 * y_std, y_mean - 3 * y_std
    ax.plot(test_x.squeeze(-1), y_mean, c="steelblue", linewidth=2)
    ax.fill_between(test_x.squeeze(-1), y_upper.squeeze(-1), y_lower.squeeze(-1),
                    alpha=0.5, facecolor="lightblue", lw=0)

def setup_axis(ax, title):
    ax.set_xlim(test_x.min()+1, test_x.max()-1.5)
    ax.set_ylim([-4, 3.5])
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params(which='both', bottom=False, top=False, labelbottom=False)
    ax.set_title(title, fontsize=18)

def save_figures(t):
    # plot inducing points
    with gpflow.params_as_tensors_for(z), gpflow.params_as_tensors_for(a):
        Z = sess.run(z.Z).squeeze(-1)
        A = sess.run(a.Z).squeeze(-1)

    fig = plt.figure(constrained_layout=True, figsize=(8, 6))
    gs = gridspec.GridSpec(4, 4, figure=fig)

    ax1 = plt.subplot(gs[:2, 1:3])
    ax2 = plt.subplot(gs[2:4, :2])
    ax3 = plt.subplot(gs[2:4, 2:])

    y_mean, y_var = solvegp.predict_f(test_x)
    plot_posterior(ax1, y_mean, y_var)
    ax1.scatter(Z, np.ones_like(Z)*(-3.5), marker="+", s=100, c="k", linewidths=1.3, label="Inducing Points")
    ax1.scatter(A, np.ones_like(A)*(-3.5), marker="^", s=100, facecolor="k", lw=0, label="Orthogonal Inducing Points")
    ax1.scatter(train_x.squeeze(-1), train_y.squeeze(-1), facecolor="k", lw=0, s=14)
    leg = ax1.legend(fontsize=14, loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=2)
    leg_texts = leg.get_texts()
    leg_texts[0].set_color("b")
    leg_texts[1].set_color("r")
    setup_axis(ax1, r"Posterior Approximation by SOLVE-GP: $p = {\color{blue} p_\|} + {\color{red} p_\perp}$")

    y_mean, y_var = solvegp.predict_fin(test_x)
    plot_posterior(ax2, y_mean, y_var)
    ax2.scatter(Z, np.ones_like(Z)*(-3.5), marker="+", s=100, c="k", linewidths=1.3)
    setup_axis(ax2, r"$\color{blue}f_\| \sim p_\|$")

    y_mean, y_var = solvegp.predict_fperp(test_x)
    plot_posterior(ax3, y_mean, y_var)
    ax3.scatter(Z, np.ones_like(Z)*(-3.5), marker="+", s=100, c="k", linewidths=1.3)
    ax3.scatter(A, np.ones_like(A)*(-3.5), marker="^", s=100, facecolor="k", lw=0)
    setup_axis(ax3, r"$\color{red}f_\perp \sim p_\perp$")

    plt.savefig("results/solvegp-iter-{}.png".format(t), bbox_inches="tight", dpi=100)
    plt.close()

## Training

Then we run the training process. We print values of the lower bound and observation variance during training.

In [10]:
save_iters = [100*i + 1 for i in range(n_iters // 100)]

for t in range(1, n_iters + 1):
    if t in save_iters:
        print("Saving figures...")
        save_figures(t)

    _, train_ll, obs_var = sess.run(
        [infer_op, obj, likelihood_var],
        feed_dict=feed_dict)
    if t % 200 == 0:
        print("Iter {}, lower bound = {:.4f}, obs_var = {:.4f}"
              .format(t, -train_ll, obs_var))

Iter 200, lower bound = -143.3596, obs_var = 0.1358
Iter 400, lower bound = -76.0936, obs_var = 0.1457
Iter 600, lower bound = -60.2646, obs_var = 0.1455
Iter 800, lower bound = -54.9912, obs_var = 0.1414
Iter 1000, lower bound = -46.6244, obs_var = 0.1359
Iter 1200, lower bound = -37.0410, obs_var = 0.1300
Iter 1400, lower bound = -51.8743, obs_var = 0.1241
Iter 1600, lower bound = -46.7503, obs_var = 0.1188
Iter 1800, lower bound = -54.1932, obs_var = 0.1142
Iter 2000, lower bound = -57.4773, obs_var = 0.1103
Iter 2200, lower bound = -51.9503, obs_var = 0.1069
Iter 2400, lower bound = -55.3193, obs_var = 0.1048
Iter 2600, lower bound = -63.9242, obs_var = 0.1030
Iter 2800, lower bound = -69.2975, obs_var = 0.1018
Iter 3000, lower bound = -28.0195, obs_var = 0.1007
Iter 3200, lower bound = -60.6562, obs_var = 0.1000
Iter 3400, lower bound = -50.5385, obs_var = 0.0995
Iter 3600, lower bound = -41.6798, obs_var = 0.0993
Iter 3800, lower bound = -66.5141, obs_var = 0.0988
Iter 4000, lowe

Finally, we generate the animation from saved figures.

In [12]:
filenames = [os.path.join("results", "solvegp-iter-{}.png".format(i)) for i in range(1, 5002, 100)]

with imageio.get_writer(os.path.join("results", 'movie.gif'), mode='I', duration=0.2) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

clip = mp.VideoFileClip(os.path.join("results", 'movie.gif'))
clip.write_videofile(os.path.join("results", 'movie.mp4'))

t:  45%|████▌     | 23/51 [00:00<00:00, 226.64it/s, now=None]

Moviepy - Building video results/movie.mp4.
Moviepy - Writing video results/movie.mp4



Moviepy - Done !
Moviepy - video ready results/movie.mp4
